In [ ]:
import cv2

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

while True:
    ret, img= cap.read()
    results = model(img, stream=True)

    cv2.imshow('Webcam', frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

NameError: name 'model' is not defined

### Small Image Filter

In [ ]:
import os

def delete_small_images(src_directory, max_size_kb=1):
    # Iterate through each subdirectory in the specified source directory
    for root, dirs, files in os.walk(src_directory):
        for file in files:
            # Check if the file is a JPG image
            if file.lower().endswith(".jpg") or file.lower().endswith(".jpeg"):
                file_path = os.path.join(root, file)

                # Check if the image size is less than the specified maximum size
                if os.path.getsize(file_path) / 1024 < max_size_kb:
                    # Delete the file
                    os.remove(file_path)
                    print(f"Deleted {file}")

# Example usage:
source_directory = "D:\\Study\\EP-ABA_BL05\\Capstone\\Capstone_MAD_Mavericks\\xyz"
delete_small_images(source_directory)


### Size Reduction

In [ ]:
import os
import time
from PIL import Image
import shutil

def reduce_and_rename(directory_path, target_size_kb=10):
    # Iterate through each subdirectory in the specified directory
    for root, dirs, files in os.walk(directory_path):
        # Sort files to ensure consistent order
        files.sort()

        # Initialize a counter for serial renaming
        serial_counter = 1

        for file in files:
            # Check if the file is a JPG image
            if file.lower().endswith(".jpg") or file.lower().endswith(".jpeg"):
                file_path = os.path.join(root, file)

                # Check if the original image is already below the target size
                if os.path.getsize(file_path) / 1024 <= target_size_kb:
                    new_file_name = f"{serial_counter}.jpg"
                    new_file_path = os.path.join(root, new_file_name)
                    os.rename(file_path, new_file_path)
                    print(f"A. Renamed {file} to {new_file_name}")

                else:
                    # Open the image using Pillow
                    with Image.open(file_path) as img:
                        # Check if either height or width is greater than 200 pixels
                        if img.width > 200 or img.height > 200:
                            # Calculate new size with a maximum dimension set to 100 pixels
                            if img.width > img.height:
                                new_size = (200, int(200 * img.height / img.width))
                            else:
                                new_size = (int(200 * img.width / img.height), 200)

                            # Resize the image while maintaining the aspect ratio
                            resized_img = img.resize(new_size, Image.ANTIALIAS)

                            # Save the resized image with quality 100, overwriting the original file
                            new_file_name = f"{serial_counter}.jpg"
                            new_file_path = os.path.join(root, new_file_name)
                            resized_img.save(new_file_path, quality=100)

                            print(f"B. Resized and renamed {file} to {new_file_name}")

                            # Close the resized image
                            resized_img.close()
                            # Delete the old file
                            os.remove(file_path)
                            print(f"Deleted old file: {file}")

                        else:
                            # If both width and height are less than or equal to 200, rename without resizing
                            img.close()
                            new_file_name = f"{serial_counter}.jpg"
                            new_file_path = os.path.join(root, new_file_name)
                            os.rename(file_path, new_file_path)
                            print(f"C. Renamed {file} to {new_file_name}")

                # Increment the serial counter
                serial_counter += 1

# Example usage:
directory_to_process = "D:\\Study\\EP-ABA_BL05\\Capstone\\Capstone_MAD_Mavericks\\xyz"
reduce_and_rename(directory_to_process)


In [ ]:
def isGoogleColab():
    import os
    try:
        import google.colab
        print("Running on Google Colab")
        from google.colab import drive  # Import the drive module
        if not os.path.exists('/content/drive'):
            drive.mount('/content/drive')
        else:
            print("Google Drive is already mounted.")
        return True
    except ImportError:
        print("Not running on Google Colab")
        return False

def isUltralytics():
  import importlib.util
  spec = importlib.util.find_spec("ultralytics")
  if spec:
    print(" Ultralytics found")
  else:
    !pip install ultralytics


In [ ]:
import os
CURRENT_PATH = os.getcwd()
if isGoogleColab():
    CURRENT_PATH = "/content/drive/Othercomputers/My Laptop/Capstone_MAD_Mavericks"

Running on Google Colab
Mounted at /content/drive


### Annotation file creation

In [ ]:
 def CMS_Generate_label(image_dir,class_name_val):
    import os
    import cv2

    # Define the directory containing your images
    image_directory = CURRENT_PATH +'/Dataset/RawData/'+ image_dir

    for image_filename in os.listdir(image_directory):
        if image_filename.endswith('.jpg'):
            # Get the full path for the original and new file
            old_file_path = os.path.join(image_directory, image_filename)
            new_file_name = os.path.join(image_directory, image_dir + "_" + image_filename)

            print(image_filename, new_file_name)

            # Rename the file
            os.rename(old_file_path, new_file_name)

    # Loop through all image files in the directory
    for image_filename in os.listdir(image_directory):
        if image_filename.endswith('.jpg'):  # Change the extension as needed
            # Create the label file name (replace the extension)
            label_filename = os.path.splitext(image_filename)[0] + '.txt'

            # Get the width and height of the image
            img = cv2.imread(os.path.join(image_directory, image_filename))
            img_height, img_width, _ = img.shape

            # Sample: Create a label file with class name, 0, 0, and image width and height
            with open(os.path.join(image_directory, label_filename), 'w') as label_file:
                label_file.write(f"{class_name_val} 0.5 0.5 1 1\n")

            print(f"Created label file for {image_filename}")


In [ ]:
import os
import random
import shutil

def CMS_split_data(inp_dir_path, out_dir_path):
    # Specify your input directory containing images and label files
    input_dir = inp_dir_path

    # Specify the output directory where you want to split the data
    out_dir = out_dir_path

    # Define the split ratios for training, validation, and testing
    train_ratio = 0.7
    val_ratio = 0.15
    test_ratio = 0.15

    # Create the necessary output directories
    train_dir = os.path.join(out_dir, 'train')
    val_dir = os.path.join(out_dir, 'valid')
    test_dir = os.path.join(out_dir, 'test')

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Get a list of all files in the input directory
    all_files = os.listdir(input_dir)

    # Shuffle the files randomly
    random.shuffle(all_files)

    # Split the files into training, validation, and testing sets
    total_files = len(all_files)
    train_split = int(total_files * train_ratio)
    val_split = int(total_files * (train_ratio + val_ratio))

    train_files = all_files[:train_split]
    val_files = all_files[train_split:val_split]
    test_files = all_files[val_split:]

    # Move image and label files to their respective directories
    for src_file in train_files:
        if src_file.endswith('.jpg'):
            base_name, _ = os.path.splitext(src_file)
            img_src = os.path.join(input_dir, src_file)
            img_dest = os.path.join(train_dir, 'images', src_file)
            label_src = os.path.join(input_dir, f'{base_name}.txt')
            label_dest = os.path.join(train_dir, 'labels', f'{base_name}.txt')

            os.makedirs(os.path.join(train_dir, 'images'), exist_ok=True)
            os.makedirs(os.path.join(train_dir, 'labels'), exist_ok=True)

            shutil.copy(img_src, img_dest)
            shutil.copy(label_src, label_dest)

    for src_file in val_files:
        if src_file.endswith('.jpg'):
            base_name, _ = os.path.splitext(src_file)
            img_src = os.path.join(input_dir, src_file)
            img_dest = os.path.join(val_dir, 'images', src_file)
            label_src = os.path.join(input_dir, f'{base_name}.txt')
            label_dest = os.path.join(val_dir, 'labels', f'{base_name}.txt')

            os.makedirs(os.path.join(val_dir, 'images'), exist_ok=True)
            os.makedirs(os.path.join(val_dir, 'labels'), exist_ok=True)

            shutil.copy(img_src, img_dest)
            shutil.copy(label_src, label_dest)

    for src_file in test_files:
        if src_file.endswith('.jpg'):
            base_name, _ = os.path.splitext(src_file)
            img_src = os.path.join(input_dir, src_file)
            img_dest = os.path.join(test_dir, 'images', src_file)
            label_src = os.path.join(input_dir, f'{base_name}.txt')
            label_dest = os.path.join(test_dir, 'labels', f'{base_name}.txt')

            os.makedirs(os.path.join(test_dir, 'images'), exist_ok=True)
            os.makedirs(os.path.join(test_dir, 'labels'), exist_ok=True)

            shutil.copy(img_src, img_dest)
            shutil.copy(label_src, label_dest)


### Check whether all Aircraft Type distributed in train+test+valid

In [ ]:
import os

def read_first_column(file_path):
    with open(file_path, 'r') as file:
        # Read the first column from each line and convert to integers
        first_column = [int(line.split()[0]) for line in file]
    return first_column

def print_unique_values(directory_path):
    unique_values = set()

    # Iterate over each file in the directory
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)

        # Ensure it's a file (not a directory) before reading
        if os.path.isfile(file_path):
            # Read the first column values and add to the set
            first_column_values = read_first_column(file_path)
            unique_values.update(first_column_values)

    # Print the unique values
    for value in sorted(unique_values):
        print(value)

# Replace 'your_directory_path' with the actual path to your directory
directory_path = 'D:\\Study\\EP-ABA_BL05\\Capstone\\Capstone_MAD_Mavericks\\Dataset\\valid\\labels'
print_unique_values(directory_path)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\Study\\EP-ABA_BL05\\Capstone\\Capstone_MAD_Mavericks\\Dataset\\valid\\labels'

### Counting no of jpg in each directory

In [ ]:
import os

main_directory = CURRENT_PATH + '/Dataset/RawData/'

total_jpg_count = 0  # Initialize the total count

# Iterate through the subdirectories
for directory_name in os.listdir(main_directory):
    directory_path = os.path.join(main_directory, directory_name)
    if os.path.isdir(directory_path):
        jpg_count = sum(1 for file in os.listdir(directory_path) if file.lower().endswith('.jpg'))
        total_jpg_count += jpg_count  # Update the total count
        print(f'"{directory_name}", = {jpg_count} .jpg files.')
        #print(f'"{directory_name}"')

# Print the total count at the end
print(f'Total count of .jpg files in all directories: {total_jpg_count}')


"A10", = 562 .jpg files.
"A400M", = 359 .jpg files.
"AG600", = 194 .jpg files.
"AV8B", = 360 .jpg files.
"B1", = 529 .jpg files.
"B2", = 404 .jpg files.
"B52", = 435 .jpg files.
"Be200", = 249 .jpg files.
"C130", = 797 .jpg files.
"C17", = 440 .jpg files.
"C2", = 688 .jpg files.
"C5", = 336 .jpg files.
"E2", = 368 .jpg files.
"E7", = 118 .jpg files.
"EF2000", = 474 .jpg files.
"F117", = 296 .jpg files.
"F14", = 437 .jpg files.
"F15", = 1089 .jpg files.
"F16", = 1135 .jpg files.
"F18", = 1134 .jpg files.
"F22", = 549 .jpg files.
"F35", = 942 .jpg files.
"F4", = 528 .jpg files.
"J20", = 442 .jpg files.
"JAS39", = 436 .jpg files.
"Mig31", = 294 .jpg files.
"Mirage2000", = 362 .jpg files.
"MQ9", = 297 .jpg files.
"P3", = 169 .jpg files.
"Rafale", = 453 .jpg files.
"RQ4", = 268 .jpg files.
"SR71", = 244 .jpg files.
"Su34", = 273 .jpg files.
"Su57", = 289 .jpg files.
"Tornado", = 366 .jpg files.
"Tu160", = 242 .jpg files.
"Tu95", = 236 .jpg files.
"U2", = 242 .jpg files.
"US2", = 558 .jpg fi

In [ ]:
def profiling_aircraft(name,id):
    CMS_Generate_label(name,id)
    CMS_split_data(CURRENT_PATH +'/Dataset/RawData/'+ name,CURRENT_PATH +'/Dataset')

In [ ]:
import os

directory_path = CURRENT_PATH +'/Dataset/RawData/'

subdirectories = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]

for i, directory_name in enumerate(subdirectories):
    if i == len(subdirectories) - 1:
        print(f'profiling_aircraft("{directory_name}", {i})')
        #print(f' \'{directory_name}\' ,') # for YAML file
    else:
        print(f'profiling_aircraft("{directory_name}", {i}),')
        #print(f' \'{directory_name}\' ,')# for YAML file



profiling_aircraft("A10", 0),
profiling_aircraft("A400M", 1),
profiling_aircraft("AG600", 2),
profiling_aircraft("AV8B", 3),
profiling_aircraft("B1", 4),
profiling_aircraft("B2", 5),
profiling_aircraft("B52", 6),
profiling_aircraft("Be200", 7),
profiling_aircraft("C130", 8),
profiling_aircraft("C17", 9),
profiling_aircraft("C2", 10),
profiling_aircraft("C5", 11),
profiling_aircraft("E2", 12),
profiling_aircraft("E7", 13),
profiling_aircraft("EF2000", 14),
profiling_aircraft("F117", 15),
profiling_aircraft("F14", 16),
profiling_aircraft("F15", 17),
profiling_aircraft("F16", 18),
profiling_aircraft("F18", 19),
profiling_aircraft("F22", 20),
profiling_aircraft("F35", 21),
profiling_aircraft("F4", 22),
profiling_aircraft("J20", 23),
profiling_aircraft("JAS39", 24),
profiling_aircraft("Mig31", 25),
profiling_aircraft("Mirage2000", 26),
profiling_aircraft("MQ9", 27),
profiling_aircraft("P3", 28),
profiling_aircraft("Rafale", 29),
profiling_aircraft("RQ4", 30),
profiling_aircraft("SR71", 31),

In [ ]:
profiling_aircraft("A10", 0),
profiling_aircraft("A400M", 1),
profiling_aircraft("AG600", 2),
profiling_aircraft("AV8B", 3),
profiling_aircraft("B1", 4),
profiling_aircraft("B2", 5),
profiling_aircraft("B52", 6),
profiling_aircraft("Be200", 7),
profiling_aircraft("C130", 8),
profiling_aircraft("C17", 9),
profiling_aircraft("C2", 10),
profiling_aircraft("C5", 11),
profiling_aircraft("E2", 12),
profiling_aircraft("E7", 13),
profiling_aircraft("EF2000", 14),
profiling_aircraft("F117", 15),
profiling_aircraft("F14", 16),
profiling_aircraft("F15", 17),
profiling_aircraft("F16", 18),
profiling_aircraft("F18", 19),
profiling_aircraft("F22", 20),
profiling_aircraft("F35", 21),
profiling_aircraft("F4", 22),
profiling_aircraft("J20", 23),
profiling_aircraft("JAS39", 24),
profiling_aircraft("Mig31", 25),
profiling_aircraft("Mirage2000", 26),
profiling_aircraft("MQ9", 27),
profiling_aircraft("P3", 28),
profiling_aircraft("Rafale", 29),
profiling_aircraft("RQ4", 30),
profiling_aircraft("SR71", 31),
profiling_aircraft("Su34", 32),
profiling_aircraft("Su57", 33),
profiling_aircraft("Tornado", 34),
profiling_aircraft("Tu160", 35),
profiling_aircraft("Tu95", 36),
profiling_aircraft("U2", 37),
profiling_aircraft("US2", 38),
profiling_aircraft("V22", 39),
profiling_aircraft("Vulcan", 40),
profiling_aircraft("XB70", 41),
profiling_aircraft("YF23", 42)

**To Check Whether Class_id for respective files are matching or not**

In [ ]:
import os

# Specify the directory where your files are located
directory = "/content/drive/Othercomputers/My Laptop/Capstone_MAD_Mavericks/Dataset/train/labels"

# Define the expected prefix and first column values for each filename
file_info = {
    "Deepak_Kumar": 2,
    "Anubhav_Bangari": 1,
    "Aakash_Luthra": 0,
}

# List all files in the directory
files = os.listdir(directory)

# Iterate over the files and check the conditions
for file_name in files:
    file_path = os.path.join(directory, file_name)

    if os.path.isfile(file_path):  # Ensure it's a file and not a directory
        #print(f"File Name: {file_name}")

        # Check if the file name starts with any of the expected prefixes
        matched_prefix = None
        for prefix, expected_value in file_info.items():
            if file_name.startswith(prefix):
                matched_prefix = prefix
                break

        if matched_prefix is not None:
            # Read the content of the file with the appropriate encoding (e.g., Latin-1)
            with open(file_path, 'r', encoding='latin-1') as file:
                for line in file:
                    # Split each line by a delimiter (e.g., space)
                    parts = line.split()

                    # Check if there is at least one element
                    if parts:
                        # Get the first column value
                        first_column_value = int(parts[0])

                        # Check if the first column value matches the expected value
                        if first_column_value == file_info[matched_prefix]:
                            #print(f"Expected first column value {expected_value} matched.")
                            asd = 1
                        else:
                            print(f"File Name: {file_name}")
                            print(f"Expected first column value {expected_value} did not match.")

        else:
            print("File name does not match any of the expected prefixes.")

        print("\n")  # Add a separator between files

In [ ]:
isGoogleColab()
isUltralytics()

Running on Google Colab
Google Drive is already mounted.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.7/645.7 kB 8.6 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import os

def isGoogleColab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

device = '0'

CURRENT_PATH = os.getcwd()

if isGoogleColab():
    from google.colab import drive
    drive.mount('/content/drive')
    CURRENT_PATH = "/content/drive/Othercomputers/My Laptop/Capstone_MAD_Mavericks"
    model = YOLO('yolov8s.pt')
else:
    model = YOLO('yolov8s.pt')

yaml_path = os.path.join(CURRENT_PATH, "Aircraft.yaml")

proj_name = "Military_Aircraft_Detection"

# Create a "runs" directory if it doesn't exist
runs_dir = os.path.join(CURRENT_PATH, "runs")
os.makedirs(runs_dir, exist_ok=True)

results = model.train(data=yaml_path, epochs=75, name=proj_name, patience=3)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ultralytics YOLOv8.0.215 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/Othercomputers/My Laptop/Capstone_MAD_Mavericks/Aircraft.yaml, epochs=75, patience=3, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=Military_Aircraft_Detection3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

train: Scanning /content/drive/Othercomputers/My Laptop/Capstone_MAD_Mavericks/Dataset/train/labels... 7840 images, 0 backgrounds, 0 corrupt:  59%|█████▉    | 7840/13231 [2:48:33<2:26:34,  1.63s/it]

In [ ]:
def isUltralytics():
  import importlib.util
  spec = importlib.util.find_spec("ultralytics")
  if spec:
    print(" Ultralytics found")
  else:
    !pip install ultralytics

isUltralytics()

from ultralytics import YOLO
import os

def isGoogleColab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

device = '0'

CURRENT_PATH = os.getcwd()

if isGoogleColab():
    from google.colab import drive
    drive.mount('/content/drive')
    CURRENT_PATH = "/content/drive/Othercomputers/My Laptop/Capstone_MAD_Mavericks"
    model = YOLO('yolov8s.pt')
else:
    model = YOLO('yolov8s.pt')

yaml_path = os.path.join(CURRENT_PATH, "Aircraft.yaml")

proj_name = "Military_Aircraft_Detection3"

# Create a "runs" directory if it doesn't exist
runs_dir = os.path.join(CURRENT_PATH, "runs")
os.makedirs(runs_dir, exist_ok=True)

weights_dir = os.path.join(runs_dir, proj_name, "weights")
last_checkpoint_path = os.path.join(weights_dir, "last.pt")

# Check if "last.pt" exists to determine whether to resume training
if os.path.exists(last_checkpoint_path):
    results = model.train(data=yaml_path, epochs=75, name=proj_name, patience=3, resume=last_checkpoint_path)
else:
    results = model.train(data=yaml_path, epochs=75, name=proj_name, patience=3)


 Ultralytics found
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ultralytics YOLOv8.0.215 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/Othercomputers/My Laptop/Capstone_MAD_Mavericks/Aircraft.yaml, epochs=75, patience=3, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=Military_Aircraft_Detection32, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, sho

train: Scanning /content/drive/Othercomputers/My Laptop/Capstone_MAD_Mavericks/Dataset/train/labels... 13231 images, 0 backgrounds, 0 corrupt: 100%|██████████| 13231/13231 [1:39:23<00:00,  2.22it/s]


train: New cache created: /content/drive/Othercomputers/My Laptop/Capstone_MAD_Mavericks/Dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/drive/Othercomputers/My Laptop/Capstone_MAD_Mavericks/Dataset/valid/labels... 2837 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2837/2837 [08:59<00:00,  5.26it/s]


val: New cache created: /content/drive/Othercomputers/My Laptop/Capstone_MAD_Mavericks/Dataset/valid/labels.cache
Plotting labels to runs/detect/Military_Aircraft_Detection32/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/Military_Aircraft_Detection32
Starting training for 75 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/75         0G     0.2952      3.527      0.991         53        640:  48%|████▊     | 393/827 [3:52:18<4:16:05, 35.40s/it]

In [ ]:
def isUltralytics():
  import importlib.util
  spec = importlib.util.find_spec("ultralytics")
  if spec:
    print(" Ultralytics found")
  else:
    !pip install ultralytics

isUltralytics()

from ultralytics import YOLO
import os

def isGoogleColab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

device = '0'

CURRENT_PATH = os.getcwd()

if isGoogleColab():
    from google.colab import drive
    drive.mount('/content/drive')
    CURRENT_PATH = "/content/drive/Othercomputers/My Laptop/Capstone_MAD_Mavericks"

yaml_path = os.path.join(CURRENT_PATH, "Aircraft.yaml")

proj_name = "Military_Aircraft_Detection"

# Create a "runs" directory if it doesn't exist
runs_dir = os.path.join(CURRENT_PATH, "runs")
os.makedirs(runs_dir, exist_ok=True)

detect_dir = os.path.join(runs_dir, "detect")
os.makedirs(detect_dir, exist_ok=True)

weights_dir = os.path.join(detect_dir, proj_name, "weights")
last_checkpoint_path = os.path.join(weights_dir, "last.pt")


# Check if "last.pt" exists to determine whether to resume training
if os.path.exists(last_checkpoint_path):
    print("Using Last check Point")
    model = YOLO(last_checkpoint_path)
    #results = model.train(data=yaml_path, epochs=2, project = detect_dir, name=proj_name, patience=2,exist_ok= True, resume=True)
    results = model.train(exist_ok= True, resume=True)
else:
    print("Fresh Start")
    model = YOLO('yolov8s.pt')
    results = model.train(data=yaml_path, epochs=2, project = detect_dir, name=proj_name, patience=2 , exist_ok= True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.7/645.7 kB 6.2 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Fresh Start


100%|██████████| 21.5M/21.5M [00:00<00:00, 166MB/s]


Ultralytics YOLOv8.0.215 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/Othercomputers/My Laptop/Capstone_MAD_Mavericks/Aircraft.yaml, epochs=2, patience=2, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/Othercomputers/My Laptop/Capstone_MAD_Mavericks/runs/detect, name=Military_Aircraft_Detection, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None,

100%|██████████| 755k/755k [00:00<00:00, 23.2MB/s]


Overriding model.yaml nc=80 with nc=44

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

train: Scanning /content/drive/Othercomputers/My Laptop/Capstone_MAD_Mavericks/Dataset/train/labels.cache... 13231 images, 0 backgrounds, 0 corrupt: 100%|██████████| 13231/13231 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/drive/Othercomputers/My Laptop/Capstone_MAD_Mavericks/Dataset/valid/labels.cache... 2837 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2837/2837 [00:00<?, ?it/s]


Plotting labels to /content/drive/Othercomputers/My Laptop/Capstone_MAD_Mavericks/runs/detect/Military_Aircraft_Detection/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000208, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /content/drive/Othercomputers/My Laptop/Capstone_MAD_Mavericks/runs/detect/Military_Aircraft_Detection
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/2         0G     0.7481      5.264      1.246         45        640:  11%|█         | 87/827 [57:03<7:57:19, 38.70s/it]

In [ ]:
# Load the TensorBoard extension
%load_ext tensorboard

# Run TensorBoard with the specified log directory
%tensorboard --logdir /content/drive/Othercomputers/My\ Laptop/Capstone_MAD_Mavericks/runs/detect/Military_Aircraft_Detection


In [3]:
def isGoogleColab():
    import os
    try:
        import google.colab
        print("Running on Google Colab")
        from google.colab import drive  # Import the drive module
        if not os.path.exists('/content/drive'):
            drive.mount('/content/drive')
        else:
            print("Google Drive is already mounted.")
        return True
    except ImportError:
        print("Not running on Google Colab")
        return False

def isUltralytics():
  import importlib.util
  spec = importlib.util.find_spec("ultralytics")
  if spec:
    print(" Ultralytics found")
  else:
    !pip install ultralytics

In [ ]:
import os




if isGoogleColab():
    BEST_PT_PATH = CURRENT_PATH + "/runs/detect/Military_Aircraft_Detection/weights/best.pt"
    TEST_IMAGE_PATH = CURRENT_PATH + "/Dataset/test/images"
else:
    BEST_PT_PATH = CURRENT_PATH + "\\runs\\detect\\Military_Aircraft_Detection\\weights\\best.pt"
    TEST_IMAGE_PATH = os.path.join("")

print(BEST_PT_PATH)
print(os.getcwd())

# Use an f-string to format the command with variable values
command = f"yolo task=detect mode=predict model={BEST_PT_PATH} conf=0.55 source={TEST_IMAGE_PATH}"

# Run the command
!{command}


Not running on Google Colab
D:\Study\EP-ABA_BL05\Capstone\Military_Aircraft_Detection_cropped_version\runs\detect\Military_Aircraft_Detection2\weights\best.pt
D:\Study\EP-ABA_BL05\Capstone\Military_Aircraft_Detection_cropped_version


C:\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
Ultralytics YOLOv8.0.188  Python-3.8.13 torch-2.0.1+cpu CPU (Intel Core(TM) i7-8550U 1.80GHz)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients

image 1/3 D:\Study\EP-ABA_BL05\Capstone\Military_Aircraft_Detection_cropped_version\Dataset\test\images\0cb13849bf51b225d1a4340a037f80b5_0.jpg: 192x640 1 A400M, 673.6ms
image 2/3 D:\Study\EP-ABA_BL05\Capstone\Military_Aircraft_Detection_cropped_version\Dataset\test\images\0cb13849bf51b225d1a4340a037f80b5_1.jpg: 192x640 1 A400M, 148.6ms
image 3/3 D:\Study\EP-ABA_BL05\Ca

In [ ]:
# Move the results to your desired Google Drive location
!mv runs/detect/predict CURRENT_PATH + "/runs/predict/"